In [1]:
import os
import sys
sys.path.append('../..')

import pytorch_lightning as pl

from lightning_modules.image_segmentation import ImageSegmentationModule
from utils.io import load_yaml


prefix = '/home/martin/Tresors/homography_imitation_learning_logs/boundary_image_segmentation/deeplabv3p/efficient_net/b0/version_0'
checkpoint = 'epoch=49-step=1499.ckpt'

config = load_yaml(os.path.join(prefix, 'config.yml'))

model = ImageSegmentationModule(**config['model'])
model = model.load_from_checkpoint(os.path.join(prefix, 'checkpoints', checkpoint), **config['model'])
model = model.eval()
model.freeze()


/home/martin/miniconda3/envs/torch19/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
import cv2
from kornia import image_to_tensor, tensor_to_image, resize

from utils.io import recursive_scan2df
from utils.sampling import ConsecutiveSequences, RandomSequences
from utils.processing import endoscopy

prefix = '/media/martin/Samsung_T5/data/endoscopic_data/cholec80_splits'
# prefix = '/media/martin/Samsung_T5/data/endoscopic_data/ROBUST_MIS_raw_data'
df = recursive_scan2df(prefix, '.mp4')
# df = recursive_scan2df(prefix, '.avi')
N = 10

paths = [os.path.join(prefix, row.folder, row.file) for _, row in df.sample(N).iterrows()]

cs = RandomSequences(max_seq=100, paths=paths, seq_len=100)
for frames in cs:
    frames = image_to_tensor(frames)
    frames = frames.to(model.device)
    frames = frames.float()/255.
    # frames = resize(frames, [270, 480])
    segs = model(frames)
    avg_seg = segs.mean(axis=0)

    for seg, frame in zip(segs, frames):
        seg = tensor_to_image(avg_seg, keepdim=False)
        frame = tensor_to_image(frame, keepdim=False)

        center, radius = endoscopy.ransacBoundaryCircle(seg*255, th=10, fit='numeric', n_pts=100, n_iter=1)

        if radius is not None:
            center, radius = center.astype(int), int(radius)
        
            top_left, shape = endoscopy.maxRectangleInCircle(seg.shape, center, radius)
            top_left, shape = top_left.astype(int), tuple(map(int, shape))
            cv2.circle(frame, (center[1], center[0]), radius, color=(1, 1, 0), thickness=2)
            cv2.rectangle(frame, (top_left[1], top_left[0]), (top_left[1]+shape[1], top_left[0]+shape[0]), (1, 1, 0), thickness=2)

            # crop = endoscopy.crop(frame, top_left, shape)
            # cv2.imshow('crop', crop)

        mask = endoscopy.bilateralSegmentation(frame*255, th=0.01)
        center, radius = endoscopy.ransacBoundaryCircle(mask, th=10, fit='numeric', n_pts=100, n_iter=1)

        if radius is not None:
            center, radius = center.astype(int), int(radius)
            cv2.circle(frame, (center[1], center[0]), radius, color=(1, 0, 1), thickness=2)

        cv2.imshow('seg', seg)
        cv2.imshow('frames', frame)
        cv2.waitKey()



NameError: name 'seg' is not defined

$1
